<a href="https://colab.research.google.com/github/SamiraDabbah1/Cloud_HW2/blob/main/Copy_of_cloud_hw2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [42]:
!pip install requests flask firebase firebase-admin beautifulsoup4 aiohttp nltk matplotlib ipywidgets aiohttp


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 12.1 MB/s eta 0:00:00


In [ ]:
import requests
from flask import Flask, request, jsonify, render_template
from firebase import firebase
from bs4 import BeautifulSoup
from urllib.parse import urljoin
from collections import Counter
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
import re
import aiohttp
import asyncio
from nltk.stem import PorterStemmer
from collections import defaultdict


app = Flask(__name__)
fbConn = firebase.FirebaseApplication('https://cloud-ex6-default-rtdb.firebaseio.com/', None)


In [39]:
def search():
    data = request.get_json()
    query = data.get('query')
    if not query:
        return jsonify({'error': 'Query not provided'})
    links = search_digitalocean(query)
    save_search_results(query, links)
    return jsonify({'query': query, 'links': links})

def save_search_results(query, links):
    fbConn.put('/search_results', query, links)

def admin():
    queries = fetch_all_queries()
    return render_template('admin.html', queries=queries)

def fetch_all_queries():
    results = fbConn.get('/search_results', None)
    return list(results.keys()) if results else []

def delete_query(query):
    fbConn.delete('/search_results', query)
    return jsonify({'success': 'Query deleted successfully'})

def delete_link(query):
    data = request.get_json()
    link_to_delete = data['link']
    links = fbConn.get('/search_results', query)
    if link_to_delete in links:
        links.remove(link_to_delete)
        fbConn.put('/search_results', query, links)
    return jsonify({'success': 'Link deleted successfully'})

def get_links(query):
    links = fbConn.get('/search_results', query)
    return jsonify(links)


def generate_statistic_chart():
    index_data = fbConn.get('/search_results', None)
    if index_data is None:
        print("No search results found in Firebase.")
        return
    word_links = Counter()
    for query, links in index_data.items():
        words = query.lower().split()
        for word in words:
            word_links[word] += len(links)
    plt.bar(word_links.keys(), word_links.values())
    plt.xlabel('Words')
    plt.ylabel('Links Count')
    plt.title('Links Count for Each Word')
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.savefig('static/chart.png')

def stats():
    generate_statistic_chart()
    return render_template('stats.html', chart='static/word_frequency_chart.png')


In [45]:

async def async_request(url):
    async with aiohttp.ClientSession() as session:
        async with session.get(url) as response:
            return await response.text()

def search_digitalocean(query, max_depth=2):

    def index_words(soup):
        index = {}
        words = re.findall(r'\w+', soup.get_text())
        for word in words:
            word = word.lower()
            if word in index:
                index[word] += 1
            else:
                index[word] = 1
        return index

    def remove_stop_words(index):
        stop_words = {'a', 'an', 'the', 'and', 'or', 'in', 'on', 'at'}
        for stop_word in stop_words:
            if stop_word in index:
                del index[stop_word]
        return index

    def apply_stemming(index):
        stemmer = PorterStemmer()
        stemmed_index = {}
        for word, count in index.items():
            stemmed_word = stemmer.stem(word)
            if stemmed_word in stemmed_index:
                stemmed_index[stemmed_word] += count
            else:
                stemmed_index[stemmed_word] = count
        return stemmed_index

    async def recursive_search_async(url, depth, visited_links=set()):
            if depth > max_depth:
                return []
            html = await async_request(url)
            soup = BeautifulSoup(html, 'html.parser')
            links = soup.find_all('a', href=True)
            found_links = []
            query_words_lower = [word.lower() for word in query.split()]
            for link in links:
                link_url = urljoin(url, link['href'])
                link_url_lower = link_url.lower()
                if link_url not in visited_links and all(word in link_url_lower for word in query_words_lower):
                    visited_links.add(link_url)
                    link_html = await async_request(link_url)
                    link_soup = BeautifulSoup(link_html, 'html.parser')
                    index = index_words(link_soup)
                    index = remove_stop_words(index)
                    index = apply_stemming(index)
                    if len(index) > 50:
                        found_links.append(link_url)
                        found_links.extend(await recursive_search_async(link_url, depth + 1, visited_links))
            return found_links

    async def search_digitalocean_async():
            return await recursive_search_async("https://www.digitalocean.com/", 0)

    loop = asyncio.new_event_loop()
    asyncio.set_event_loop(loop)
    return loop.run_until_complete(search_digitalocean_async())


In [47]:
from IPython.display import HTML
from google.colab import output

class WelcomePage:
    def __init__(self):
        self.welcomeHTML = '''
        <!DOCTYPE html>
        <html lang="en">
        <head>
          <meta charset="UTF-8">
          <meta name="viewport" content="width=device-width, initial-scale=1.0">
          <title>Welcome</title>
          <!-- Include Tailwind CSS CDN -->
          <link href="https://cdn.jsdelivr.net/npm/tailwindcss@2.2.19/dist/tailwind.min.css" rel="stylesheet">
          <!-- Include Font Awesome CDN -->
          <link rel="stylesheet" href="https://cdnjs.cloudflare.com/ajax/libs/font-awesome/6.5.1/css/all.min.css"
            integrity="sha512-DTOQO9RWCH3ppGqcWaEA1BIZOC6xxalwEsw9c2QQeAIftl+Vegovlnee1c9QX4TctnWMn13TZye+giMm8e2LwA=="
            crossorigin="anonymous" referrerpolicy="no-referrer" />
        </head>
        <body class="overflow-hidden">
            <div class="w-screen h-screen overflow-hidden relative before:block before:absolute before:bg-black before:h-full before:w-full before:top-0 before:left-0 before:z-10 before:opacity-30" style="background-image: url('https://ditto-oceandecade.org/wp-content/uploads/2023/04/Group-25-min.png'); background-size: cover; background-repeat: no-repeat; background-position: center center; color: #FFA500; text-align: center; height: 550px; display: flex; flex-direction: column; justify-content: center;">
                <div class="relative z-20 max-w-screen-lg mx-auto grid grid-cols-12 h-full items-center">
                  <div class="col-span-12 text-left"> <!-- Change to text-left -->
                    <span class="uppercase text-white text-xl font-bold mb-2 mt-4 block">Welcome to EducCloud</span>
                    <h1 class="text-white font-extrabold text-4xl mb-8">Start studying from home. Arrange your space, tools, and mindset.</h1>
                    <p class="text-slate-500 text-lg text-blue-200">
                        As experts in the field, we advocate providing student-focused support, even when we have a lot of work.
                    </p>
                    <div class="mt-6 flex gap-8">
                        <button onclick="renderLogin()" class="bg-blue-200 hover:bg-blue-500 text-black uppercase py-3 px-6 rounded-full transition duration-300">Login</button>
                        <button onclick="renderSignUp()" class="bg-blue-200 hover:bg-blue-500 text-black uppercase py-3 px-6 rounded-full transition duration-300">Sign up</button>
                    </div>
                  </div>
                </div>
              </div>

            <script>
                function renderLogin() {
                    google.colab.kernel.invokeFunction('notebook.render_login', []);
                }

                function renderSignUp() {
                    google.colab.kernel.invokeFunction('notebook.render_sign_up', []);
                }


            </script>
        </body>
        </html>
        '''

    def display(self):
        return HTML(self.welcomeHTML)

def render_login():
    loginHTML = '''
    <!DOCTYPE html>
    <html lang="en">

    <head>
      <meta charset="UTF-8">
      <meta name="viewport" content="width=device-width, initial-scale=1.0">
      <title>Sign IN</title>
      <!-- Include Tailwind CSS CDN -->
      <link href="https://cdn.jsdelivr.net/npm/tailwindcss@2.2.19/dist/tailwind.min.css" rel="stylesheet">
      <!-- Include Font Awesome CDN -->
      <link rel="stylesheet" href="https://cdnjs.cloudflare.com/ajax/libs/font-awesome/6.5.1/css/all.min.css"
        integrity="sha512-DTOQO9RWCH3ppGqcWaEA1BIZOC6xxalwEsw9c2QQeAIftl+Vegovlnee1c9QX4TctnWMn13TZye+giMm8e2LwA=="
        crossorigin="anonymous" referrerpolicy="no-referrer" />

      <!-- Custom Styles -->
      <style>
        body {
          background-image: url('https://marine.copernicus.eu/sites/default/files/images/news/Sans-titre-26.png');
          background-size: cover;
          background-repeat: no-repeat;
          background-position: center center;
          color: #FFA500;
          text-align: center;
          height: 550px;
          display: flex;
          flex-direction: column;
          justify-content: center;
        }

        /* Style input fields */
        input {
          background-color: white;
        }
      </style>
    </head>

    <body class="bg-gray-100 flex items-center justify-center h-screen">
      <img src="https://marine.copernicus.eu/sites/default/files/images/news/Sans-titre-26.png" alt="Sign-in image"
        class="w-full h-full object-cover" style="width: 100vw; height: 100vh; object-fit: cover; position: fixed; top: 0; left: 0; z-index: -1;"
        alt="">
      <div class="w-full sm:w-96 max-w-md bg-white shadow-md rounded px-8 py-6 mx-auto">
        <div class="sm:mx-auto sm:w-full sm:max-w-sm">
          <h2 class="mt-4 text-2xl font-bold leading-9 tracking-tight text-blue-800">Sign in</h2>
        </div>
        <div class="mt-6 sm:mx-auto sm:w-full sm:max-w-sm">
          <form id="loginForm" class="space-y-6" method="POST" action="/user/login">
            <div>
              <label for="email" class="block text-sm font-medium leading-6 text-blue-800">Email address</label>
              <div class="mt-2">
                <input id="emailLogin" name="email" type="email" autocomplete="email" required
                  class="block w-full rounded-md border-0 py-1.5 text-gray-900 shadow-sm ring-1 ring-inset ring-gray-400 placeholder:text-gray-500 focus:ring-2 focus:ring-inset focus:ring-indigo-800 sm:text-sm sm:leading-6"
                  style="margin-bottom: 1rem;">
              </div>
            </div>
            <div>
              <label for="password" class="block text-sm font-medium leading-6 text-blue-800">Password</label>
              <div class="mt-2">
                <input id="passwordLogin" name="password" type="password" autocomplete="current-password" required
                  class="block w-full rounded-md border-0 py-1.5 text-gray-900 shadow-sm ring-1 ring-inset ring-gray-400 placeholder:text-gray-500 focus:ring-2 focus:ring-inset focus:ring-indigo-800 sm:text-sm sm:leading-6"
                  style="margin-bottom: 2rem;">
              </div>
            </div>
            <div>
            <a href="javascript:void(0)" onclick="renderAboutUs()" class="block w-full">
                <button type="button" class="flex w-full justify-center rounded-md bg-blue-800 px-3 py-1.5 text-sm font-semibold leading-6 text-white shadow-sm hover:bg-indigo-600 focus-visible:outline focus-visible:outline-2 focus-visible:outline-offset-2 focus-visible:outline-indigo-700">
                    Sign in
                </button>
            </a>
        </div>

          </form>
          <p class="mt-6 text-center text-sm text-gray-700">
          Not a member? <a href="javascript:void(0)" onclick="renderSignUp()"
          class="font-semibold leading-6 text-indigo-800 hover:text-indigo-00">Sign up</a>
        </p>
        </div>
      </div>
      <script>
      function renderSignUp() {
        google.colab.kernel.invokeFunction('notebook.render_sign_up', []);
        }

     function renderAboutUs() {
        google.colab.kernel.invokeFunction('notebook.render_about_us', []);
                }

      </script>

    </body>

    </html>


    '''
    output.eval_js('google.colab.output.setIframeHeight(0, true);')
    output.clear()
    display(HTML(loginHTML))

def render_sign_up():
    signUpHTML = '''
   <!DOCTYPE html>
    <html lang="en">

    <head>
      <meta charset="UTF-8">
      <meta name="viewport" content="width=device-width, initial-scale=1.0">
      <title>Sign up</title>
      <!-- Include Tailwind CSS CDN -->
      <link href="https://cdn.jsdelivr.net/npm/tailwindcss@2.2.19/dist/tailwind.min.css" rel="stylesheet">
      <!-- Include Font Awesome CDN -->
      <link rel="stylesheet" href="https://cdnjs.cloudflare.com/ajax/libs/font-awesome/6.5.1/css/all.min.css"
        integrity="sha512-DTOQO9RWCH3ppGqcWaEA1BIZOC6xxalwEsw9c2QQeAIftl+Vegovlnee1c9QX4TctnWMn13TZye+giMm8e2LwA=="
        crossorigin="anonymous" referrerpolicy="no-referrer" />
      <!-- Custom Styles -->
      <style>
        body {
          background-image: url('https://marine.copernicus.eu/sites/default/files/images/news/Sans-titre-26.png');
          background-size: cover;
          background-repeat: no-repeat;
          background-position: center center;
          color: #FFA500;
          text-align: center;
          height: 550px;
          display: flex;
          flex-direction: column;
          justify-content: center;
        }

        .register {
          background-color: white;
        }
      </style>
    </head>

    <body class="bg-gray-100 flex items-center justify-center h-screen">
      <img src="https://marine.copernicus.eu/sites/default/files/images/news/Sans-titre-26.png" style="width: 100vw; height: 100vh; object-fit: cover; position: fixed; top: 0; left: 0; z-index: -1;" alt="">

      <!-- Register section -->
      <div class="form-container mt-10">
        <form class="register max-w-md mx-auto bg-white shadow-md rounded px-8 pt-6 pb-8 mb-4" action="#" method="post"
          enctype="multipart/form-data" onsubmit="return handleRegisterSubmit(this);">
          <h2 class="mt-6 mb-6 text-center text-3xl font-bold leading-9 tracking-tight text-blue-800">Create Account</h2>

          <div class="mb-4 flex flex-wrap">
            <label class="block uppercase tracking-wide text-blue-800 text-s font-bold" for="name">Your Name</label>
            <input
              class="appearance-none block w-full bg-white border-gray-400 text-gray-700 border rounded py-3 px-4 leading-tight focus:outline-none focus:bg-white"
              id="name" name="name" type="text" placeholder="Enter your name" maxlength="50">
          </div>

          <div class="mb-4 flex flex-wrap">
            <label class="block uppercase tracking-wide text-blue-800 text-s font-bold" for="email">Your Email</label>
            <input
              class="appearance-none block w-full bg-white border-gray-400 text-gray-700 border rounded py-3 px-4 leading-tight focus:outline-none focus:bg-white focus:border-gray-500"
              id="emailRegister" name="email" type="email" placeholder="Enter your email" maxlength="20">
          </div>

          <div class="mb-4 flex flex-wrap">
            <label class="block uppercase tracking-wide text-blue-800 text-s font-bold" for="password">Your Password</label>
            <input
              class="appearance-none block w-full bg-white border-gray-400 text-gray-700 border rounded py-3 px-4 leading-tight focus:outline-none focus:bg-white focus:border-gray-500"
              id="passwordRegister" name="password" type="password" placeholder="Enter your password" maxlength="20">
          </div>

          <div class="mb-4 flex flex-wrap">
            <label class="block uppercase tracking-wide text-blue-800 text-s font-bold" for="cpass">Confirm Password</label>
            <input
              class="appearance-none block w-full bg-white border-gray-400 text-gray-700 border rounded py-3 px-4 leading-tight focus:outline-none focus:bg-white focus:border-gray-500"
              id="cpass" name="cpass" type="password" placeholder="Confirm your password" maxlength="20">
          </div>

          <div>
               <a href="javascript:void(0)" onclick="renderLogin()">
                  <button type="submit" name="submit" class="flex w-full justify-center rounded-md bg-blue-800 px-3 py-1.5 text-m font-semibold leading-6 text-white shadow-sm hover:bg-indigo-600 focus-visible:outline focus-visible:outline-2 focus-visible:outline-offset-2 focus-visible:outline-indigo-800 mb-6">
                      Sign up
                  </button>
              </a>
          </div>


           <p class="text-center text-gray-700 text-sm">
            Already have an account?
            <a href="javascript:void(0)" onclick="renderLogin()" class="text-indigo-800 font-semibold">Login</a>
          </p>
        </form>
      </div>

      <script>

      function renderLogin() {
                google.colab.kernel.invokeFunction('notebook.render_login', []);
      }

      </script>

    </body>

    </html>

    '''
    output.eval_js('google.colab.output.setIframeHeight(0, true);')
    output.clear()
    display(HTML(signUpHTML))


def render_about_us():
# about us page
    AboutUSHTML = \
    '''
    <!DOCTYPE html>
    <html lang="en">
    <head>
      <meta charset="UTF-8">
      <meta name="viewport" content="width=device-width, initial-scale=1.0">
      <title>About US</title>
      <link href="https://cdn.jsdelivr.net/npm/tailwindcss@2.2.19/dist/tailwind.min.css" rel="stylesheet">
      <link rel="stylesheet" href="https://cdnjs.cloudflare.com/ajax/libs/font-awesome/6.5.1/css/all.min.css"
        integrity="sha512-DTOQO9RWCH3ppGqcWaEA1BIZOC6xxalwEsw9c2QQeAIftl+Vegovlnee1c9QX4TctnWMn13TZye+giMm8e2LwA=="
        crossorigin="anonymous" referrerpolicy="no-referrer" />
      <link href="https://fonts.googleapis.com/css2?family=Poppins:wght@400;500;600;700&display=swap" rel="stylesheet">
      <style>
        body {
          font-family: 'CustomFont', sans-serif;
          transition: background-color 0.5s ease, color 0.5s ease;
          background-color: #ffffff;
          color: #ffffff;
          margin: 0;
          padding: 0;
        }

        body.theme-dark {
          background-color: #1A202C;
          color: #EDF2F7;
        }

        h1 {
          font-size: 2rem;
          margin-top: 0;
        }

        .button {
          padding: 5px 10px;
          font-size: 12px;
          color: #ffffff;
          border: none;
          border-radius: 5px;
          cursor: pointer;
          margin-right: 8px;
          text-shadow: 1px 1px 2px rgba(0, 0, 0, 0.5);
      }


        header {
          background-color: #1d3a5f;
          padding: 20px;
          color: #ffffff;
          display: flex;
          align-items: center;
          justify-content: space-between;
          border: none;
          border-radius: 0;
        }

        main {
          padding: 20px;
        }

        footer {
          text-align: center;
          background-color: #1d3a5f;
          color: #ffffff;
          padding: 20px;
          position: fixed;
          bottom: 0;
          width: 100%;
        }

        .about {
          padding: 8px;
        }

        .row {
          display: flex;
          align-items: center;
        }

        .image img {
          max-width: 100%;
        }

        .content {
          flex-grow: 1;
        }

        .text-blue-500 {
          color: #3869aa;
        }

        .bg-blue-500 {
          background-color: #3869aa;
        }

        .hover\:bg-blue-600:hover {
          background-color: #0056b3;
        }

        .py-2 {
          padding-top: 0.5rem;
          padding-bottom: 0.5rem;
        }

        .rounded-xl {
          border-radius: 0.75rem;
        }

        .shadow-lg {
          box-shadow: 0 4px 6px -1px rgba(0, 0, 0, 0.1), 0 2px 4px -1px rgba(0, 0, 0, 0.06);
        }

        .text-2xl {
          font-size: 1.5rem;
        }

        .fixed {
          position: fixed;
        }

        .bottom-0 {
          bottom: 0;
        }

        .w-full {
          width: 100%;
        }

            footer {
                text-align: center;
                background-color: #1d3a5f;
                color: #ffffff;
                padding: 20px;
                position: fixed;
                bottom: 0;
                width: 100%;
            }

        .about {
          display: flex;
          align-items: center;
        }

        .about .image {
          flex: 1;
        }

        .about .content {
          flex: 1;
          padding: 0 8px;
        }

        .about .image img {
          width: 100%;
          height: auto;
        }
      </style>
    </head>
    <body>
      <header>
        <h1 class="text-white">EducCloud</h1>
        <div class="flex items-center">
         <a href="javascript:void(0)" class="button mr-2" onclick="GameHOMEHTML()">Home</a>
         <a href="javascript:void(0)" class="button mr-2" onclick="renderAboutUs()">About Us</a>
         <a href="javascript:void(0)" class="button mr-2" onclick="renderContactUs()">Contact Us</a>
         <a href="javascript:void(0)" class="button mr-2" onclick="renderSearch()">Search</a>
         <a href="javascript:void(0)" class="button mr-2" onclick="renderStatistics()">Statistics</a>
         <a href="javascript:void(0)" class="button mr-2" onclick="renderAdmin()">Admin</a>
         <a href="javascript:void(0)" id="logoutButton" class="button flex items-center" onclick="logoutUser(); return confirm('Are you sure you want to log out?');">
          <i class="fas fa-sign-out-alt mr-2"></i> Logout
        </a>
        <button id="darkModeToggle" class="button"><i class="fas fa-moon"></i></button>
      </header>

      <main>
        <section class="about p-8">
          <div class="row flex items-center">
            <div class="image w-1/2">
              <img src="https://encrypted-tbn2.gstatic.com/images?q=tbn:ANd9GcQ0yYC0UuXQ12ZgzeGGZNQeGFVDae4rEAdNEAaE9S1nTbxoDbpg" alt="About Image">
            </div>
            <div class="content w-1/2 ml-8">
              <h3 class="text-3xl font-semibold mb-4 text-blue-500">Why Choose Us?</h3>
              <p class="text-gray-700">
                Our EducCloud is designed to provide a seamless and efficient online learning experience
                for both educators and learners. With advanced features and user-friendly interface,
                our platform aims to revolutionize the way education is delivered and accessed.
              </p>
              <a href="javascript:void(0)" id="gameLink" class="mt-4 text-xl w-full text-white bg-blue-500 hover:bg-blue-600 py-2 rounded-xl shadow-lg block text-center" onclick="GameHOMEHTML()">EducCloud Game</a>
            </div>
          </div>
        </section>
      </main>

      <footer>
        <div class="footer-content">
          <p>&copy; 2024 Cloud Computing Learning. All rights reserved.</p>
        </div>
      </footer>
      <script>
          darkModeToggle.addEventListener('click', () => {
            document.body.classList.toggle('theme-dark');
            if (document.body.classList.contains('theme-dark')) {
                darkModeToggle.innerHTML = '<i class="fas fa-sun"></i>';
            } else {
                darkModeToggle.innerHTML = '<i class="fas fa-moon"></i>';
            }
        })

        function logoutUser() {
        google.colab.kernel.invokeFunction('notebook.render_login', []);
        }

       function renderAboutUs() {
        google.colab.kernel.invokeFunction('notebook.render_about_us', []);
                }

        function renderContactUs() {
          google.colab.kernel.invokeFunction('notebook.renderContactUs', []);
                }


        function renderLogout() {
        google.colab.kernel.invokeFunction('notebook.render_logout', []);
                }

        function GameHOMEHTML() {
        google.colab.kernel.invokeFunction('notebook.GameHOMEHTML', []);
         }

        function renderSearch() {
          google.colab.kernel.invokeFunction('notebook.renderSearch', []);
        }

        function renderStatistics() {
          google.colab.kernel.invokeFunction('notebook.renderStatistics', []);
        }

        function renderAdmin() {
          google.colab.kernel.invokeFunction('notebook.renderAdmin', []);
        }

      </script>
    </body>
    </html>
    '''
    output.eval_js('google.colab.output.setIframeHeight(0, true);')
    output.clear()
    display(HTML(AboutUSHTML))



def renderContactUs():
# Contact US page
  renderContactUs = \
'''
<!DOCTYPE html>
<html lang="en">

<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Courses</title>
    <link href="https://cdn.jsdelivr.net/npm/tailwindcss@2.2.19/dist/tailwind.min.css" rel="stylesheet">
    <link rel="stylesheet" href="https://cdnjs.cloudflare.com/ajax/libs/font-awesome/6.5.1/css/all.min.css"
        integrity="sha512-DTOQO9RWCH3ppGqcWaEA1BIZOC6xxalwEsw9c2QQeAIftl+Vegovlnee1c9QX4TctnWMn13TZye+giMm8e2LwA=="
        crossorigin="anonymous" referrerpolicy="no-referrer" />
    <link href="https://fonts.googleapis.com/css2?family=Poppins:wght@400;500;600;700&display=swap" rel="stylesheet">
    <style>
        body {
            font-family: 'Poppins', Arial, sans-serif;
            transition: background-color 0.5s ease, color 0.5s ease;
            background-color: rgba(115, 158, 233, 0.4);
            color: #2b2b2b;
            margin: 0;
            padding: 0;
        }

        h1 {
            font-size: 2rem;
            margin-top: 0;
        }

        .button {
          padding: 5px 10px;
          font-size: 12px;
          color: #ffffff;
          border: none;
          border-radius: 5px;
          cursor: pointer;
          margin-right: 8px;
          text-shadow: 1px 1px 2px rgba(0, 0, 0, 0.5);
        }

        header {
            background-color: #1d3a5f;
            padding: 20px;
            color: #ffffff;
            display: flex;
            align-items: center;
            justify-content: space-between;
            border: none;
            border-radius: 0;
        }

        main {
            padding: 20px;
        }

        footer {
            text-align: center;
            background-color: #1d3a5f;
            color: #ffffff;
            padding: 20px;
            position: fixed;
            bottom: 0;
            width: 100%;
        }

        .theme-dark {
            background-color: #1A202C;
            color: #EDF2F7;
        }

        .about {
            padding: 8px;
        }

        .row {
            display: flex;
            align-items: center;
        }

        .image img {
            max-width: 100%;
        }

        .content {
            flex-grow: 1;
        }

        .text-blue-500 {
            color: #3869aa;
        }

        .bg-blue-500 {
            background-color: #3869aa;
        }

        .hover\:bg-blue-600:hover {
            background-color: #0056b3;
        }

        .py-2 {
            padding-top: 0.5rem;
            padding-bottom: 0.5rem;
        }

        .rounded-xl {
            border-radius: 0.5rem;
        }

        .shadow-lg {
            box-shadow: 0 4px 6px -1px rgba(0, 0, 0, 0.1), 0 2px 4px -1px rgba(0, 0, 0, 0.06);
        }

        .text-2xl {
            font-size: 1.5rem;
        }

        .fixed {
            position: fixed;
        }

        .bottom-0 {
            bottom: 0;
        }

        .w-full {
            width: 100%;
        }

        .about {
            display: flex;
            align-items: center;
        }

        .about .image {
            flex: 1;
        }

        .about .content {
            flex: 1;
            padding: 0 8px;
        }

        .about .image img {
            width: 100%;
            height: auto;
        }

        /* Updated style for Contact Information */
        .contact-info {
            background-color: #ffffff;
            padding: 16px;
            border-radius: 10px;
        }
    </style>
    </head>

    <body>
      <header>
        <h1 class="text-white">EducCloud</h1>
        <div class="flex items-center">
         <a href="javascript:void(0)" class="button mr-2" onclick="GameHOMEHTML()">Home</a>
         <a href="javascript:void(0)" class="button mr-2" onclick="renderAboutUs()">About Us</a>
         <a href="javascript:void(0)" class="button mr-2" onclick="">Contact Us</a>
         <a href="javascript:void(0)" class="button mr-2" onclick="renderSearch()">Search</a>
         <a href="javascript:void(0)" class="button mr-2" onclick="renderStatistics()">Statistics</a>
         <a href="javascript:void(0)" class="button mr-2" onclick="renderAdmin()">Admin</a>
         <a href="javascript:void(0)" id="logoutButton" class="button flex items-center" onclick="logoutUser(); return confirm('Are you sure you want to log out?');">
          <i class="fas fa-sign-out-alt mr-2"></i> Logout
        </a>
        <button id="darkModeToggle" class="button"><i class="fas fa-moon"></i></button>
      </header>

  <main>
        <section class="py-8">
            <h1 class="text-2xl font-bold mb-4">Contact Us</h1>

            <div class="grid grid-cols-1 md:grid-cols-2 gap-8">
                <!-- Contact Form -->
                <div class="rounded-xl shadow-lg p-6 bg-white">
                    <h2 class="text-lg font-semibold mb-4">Send us a Message</h2>

                    <form>
                        <div class="mb-4">
                            <label for="name" class="block text-sm font-medium text-gray-700">Your Name</label>
                            <input type="text" id="name" name="name" class="mt-1 p-2 w-full border rounded-md border-gray-300" required>
                        </div>

                        <div class="mb-4">
                            <label for="email" class="block text-sm font-medium text-gray-700 color: #FFFFFF">Your Email</label>
                            <input type="email" id="email" name="email" class="mt-1 p-2 w-full border rounded-md border-gray-300" required>
                        </div>

                        <div class="mb-4">
                            <label for="message" class="block text-sm font-medium text-gray-700">Message</label>
                            <textarea id="message" name="message" rows="4"
                                class="mt-1 p-2 w-full border rounded-md" required></textarea>
                        </div>

                        <button type="submit" class="button bg-blue-500 hover:bg-blue-600">Submit</button>
                    </form>
                </div>

                <!-- Contact Information -->
                <div class="contact-info rounded-xl shadow-lg p-6">
                    <h2 class="text-lg font-semibold mb-4">Contact Information</h2>

                    <div class="mb-4">
                        <p class="text-gray-600"><strong>Email:</strong> <a
                                href="mailto:info@educcloud.com">unicorn@cloud.com</a></p>
                    </div>

                    <div class="mb-4">
                        <p class="text-gray-600"><strong>Phone:</strong> <a
                                href="tel:+11234567890">123 456-7890</a></p>
                    </div>

                    <div class="mb-4">
                        <p class="text-gray-600"><strong>Address:</strong> 123 Deir al-Asad </p>
                    </div>

                    <div class="mb-4">
                        <p class="text-gray-600"><strong>Map:</strong> <a
                                href="https://maps.app.goo.gl/no4vRcNWGn6y67zLA"
                                target="_blank">View on Google Map</a></p>
                        <!-- Replace "YourLocation" with the actual link to your Google Map location -->
                    </div>
                    <!-- You can add more contact information as needed -->
                </div>
            </div>
        </section>
    </main>

    <footer>
        <div class="footer-content">
            <p>&copy; 2024 Cloud Computing Learning. All rights reserved.</p>
        </div>
    </footer>
    <script>
        darkModeToggle.addEventListener('click', () => {
            document.body.classList.toggle('theme-dark');
            if (document.body.classList.contains('theme-dark')) {
                darkModeToggle.innerHTML = '<i class="fas fa-sun"></i>';
            } else {
                darkModeToggle.innerHTML = '<i class="fas fa-moon"></i>';
            }
        })

        function logoutUser() {
        google.colab.kernel.invokeFunction('notebook.render_login', []);
        }

       function renderAboutUs() {
        google.colab.kernel.invokeFunction('notebook.render_about_us', []);
                }

        function renderContactUs() {
        google.colab.kernel.invokeFunction('notebook.renderContactUs', []);
                }


        function renderLogout() {
        google.colab.kernel.invokeFunction('notebook.render_logout', []);
                }

        function GameHOMEHTML() {
        google.colab.kernel.invokeFunction('notebook.GameHOMEHTML', []);
         }

        function renderSearch() {
          google.colab.kernel.invokeFunction('notebook.renderSearch', []);
        }

        function renderStatistics() {
          google.colab.kernel.invokeFunction('notebook.renderStatistics', []);
        }

        function renderAdmin() {
          google.colab.kernel.invokeFunction('notebook.renderAdmin', []);
        }


      </script>
    </body>
   </html>

'''
  output.eval_js('google.colab.output.setIframeHeight(0, true);')
  output.clear()
  display(HTML(renderContactUs))

def GameHOMEHTML():
  #homePage
  GameHOMEHTML = \
'''
<!DOCTYPE html>
<html lang="en">

<head>
  <meta charset="UTF-8">
  <meta name="viewport" content="width=device-width, initial-scale=1.0">
  <title>Game Home</title>
  <link href="https://cdn.jsdelivr.net/npm/tailwindcss@2.2.19/dist/tailwind.min.css" rel="stylesheet">
  <link rel="stylesheet" href="https://cdnjs.cloudflare.com/ajax/libs/font-awesome/6.5.1/css/all.min.css"
    integrity="sha512-DTOQO9RWCH3ppGqcWaEA1BIZOC6xxalwEsw9c2QQeAIftl+Vegovlnee1c9QX4TctnWMn13TZye+giMm8e2LwA=="
    crossorigin="anonymous" referrerpolicy="no-referrer" />
  <link href="https://fonts.googleapis.com/css2?family=Poppins:wght@400;500;600;700&display=swap" rel="stylesheet">
  <style>
    body {
      background-image: url('https://doimages.nyc3.cdn.digitaloceanspaces.com/002Blog/DO-Gaming-Premium-CPUOptimized.png');
      background-size: cover;
      background-repeat: no-repeat;
      background-position: center center;
      font-family: 'CustomFont', sans-serif;
      font-size: 18px;
      height: 570px;
      display: flex;
      align-items: center;
      justify-content: center;
      text-align: center;
      flex-direction: column;
    }
    #screen {
      display: flex;
      flex-direction: column;
      align-items: center;
      justify-content: center;
      padding: 60px;
      background-color: rgba(255, 255, 255, 0.4);
      border-radius: 10px;
      margin: 0 auto;
      position: absolute;
      top: 50%;
      left: 50%;
      transform: translate(-50%, -50%);
      max-width: 800px;
      width: 100%;
    }

    #userScore,
    #maxQuestionInARow,
    #userWelcome {
      color: black;
      text-shadow: 2px 2px 5px white;
      font-weight: bold;
      font-size: 20px;
    }

    #leadersTable,
    #gameInstruction {
      display: inline-block;
      margin-right: 15px;
      padding: 10px 20px;
      background-color: #1d3a5f;
      color: white;
      border: none;
      border-radius: 5px;
      font-size: 16px;
      cursor: pointer;
      font-family: 'CustomFont', sans-serif;
      margin-bottom: 45px;
    }


    #numberQuestionLabel {
      color: black;
      text-shadow: 2px 2px 5px white;
      font-weight: bold;
      font-size: 18px;
      margin-top: 30px;
    }

    #selectNumberOfQuestion,
    #startBtn {
      margin-top: 17px;
    }

    button {
      padding: 10px 20px;
      background-color: #1d3a5f;
      color: white;
      border: none;
      border-radius: 5px;
      font-size: 16px;
      cursor: pointer;
      font-family: 'CustomFont', sans-serif;
    }

    #logoutBtn {
      margin-top: 30px;
    }

    #startBtn {
      width: 150px;
      height: 150px;
      border-radius: 50%;
      font-size: 24px;
      box-shadow: 0 0 10px rgba(0, 0, 0, 0.3);
    }

    #selectNumberOfQuestion {
      font-size: 16px;
      font-family: 'CustomFont', sans-serif;
      width: 60px;
      background-color: #383838;
      color: white;
    }

    .helpModal {
      display: none;
      position: fixed;
      z-index: 1;
      left: 0;
      top: 0;
      width: 100%;
      height: 100%;
      overflow: auto;
      background-color: rgba(0, 0, 0, 0.7);
    }

    .helpModal-content {
      background-color: #FFFFFF;
      margin: 15% auto;
      padding: 20px;
      border: 1px solid #888;
      width: 60%;
      box-shadow: 0 0 10px rgba(0, 0, 0, 0.3);
      color: #3f77c0;
      font-family: 'CustomFont', sans-serif;
    }

    .close {
      color: #888;
      float: right;
      font-size: 28px;
      font-weight: bold;
      cursor: pointer;
    }

    #popuptitle {
      font-weight: bold;
      font-size: 18px;
      margin-bottom: 10px;
    }

    #popupContent {
      margin-bottom: 10px;
    }

    #badgeDiv {
      display: inline-block;
      margin-right: -350px;
      float: left;
      margin-top: 10px;
    }

    button#startBtn:hover,
    button#logoutBtn:hover,
    button#leadersTable:hover,
    button#gameInstruction:hover {
      background-color: #3f77c0;
    }

    button#startBtn:active,
    button#logoutBtn:active,
    button#leadersTable:active,
    button#gameInstruction:active {
      background-color: #3869aa;
    }
  </style>
</head>

<body>
  <div id='screen'>
    <div>
      <button id='leadersTable'>Leaders Table</button>
      <button id="gameInstruction">Game Instruction</button>
    </div>
    <div id='divUserInfo'></div>
    <label id='numberQuestionLabel'>Choose number of questions for this session</label>
    <select id='selectNumberOfQuestion' onchange="selectNumberOfQuestion()">
      <option value='5'>5</option>
      <option value='10'>10</option>
      <option value='15'>15</option>
      <option value='Endless'>&infin;</option>
    </select>
    <button id='startBtn'>Start</button>
    <button id='logoutBtn'><i class="fas fa-sign-out-alt mr-2"></i>Logout</button>
    <div id="helpModal" class="helpModal">
      <div class="helpModal-content">
        <span class="close" onclick="hideHelpModal()">&times;</span>
        <p id="popuptitle">How To Play:</p>
        <p id="popupContent">&bull; You have a wide variety of multiple choice questions on cloud computing.</p>
        <p id="popupContent">&bull; The more questions you answer correctly, the more points you earn. <br>Reaching the
          first three places will earn you a badge according to your location.</p>
        <p id="popupContent">&bull; You will be able to see your ranking among all users (by clicking on Leader
          table).</p>
        <p id="popupContent">&bull; Before each game you can choose the desired amount of questions for the current
          session, after choosing you can start the game by pressing the start button.<br><br><br></p>
        <p id="popupContent" style="font-weight: bold; font-size: 18px;">How to get points:</p>
        <p id="popupContent">&bull; Answering an easy question will earn you 5 points, a medium one 10 and a difficult
          one 15.<br> For each sequence of 3 correct answers you will win 3 points.<br></p>
        <p id="popupContent">&bull; Note: medium questions will appear only if you have at least 70 scores<br> and hard
          questions will appear if you have at least 150 scores.<br><br><br></p>
        <p id="popupContent" style="font-weight: bold;">Are you ready to be the best cloud computing CEO in the
          city?</p>
        <p id="popupContent" style="font-weight: bold;">Have fun <i class="fas fa-smile mr-2"></i></p>
      </div>
    </div>
  </div>
  <script>
    var helpModal = document.getElementById("helpModal");
    document.getElementById("gameInstruction").onclick = function () {
      helpModal.style.display = "block";
    };

    function hideHelpModal() {
      helpModal.style.display = "none";
    }

    document.querySelector('#startBtn').onclick = () => {
      google.colab.kernel.invokeFunction('startGame', [], {});
    };
    document.querySelector('#logoutBtn').onclick = () => {
      google.colab.kernel.invokeFunction('notebook.render_sign_up', []);

    };
    document.querySelector('#leadersTable').onclick = () => {
      google.colab.kernel.invokeFunction('notebook.leaderTableHTML', []);
    };

    function selectNumberOfQuestion() {
      var selectElement = document.getElementById("selectNumberOfQuestion");
      var selectedOption = selectElement.options[selectElement.selectedIndex].value;
      google.colab.kernel.invokeFunction('setQuestionNumberForGame', ['0', selectedOption], {
        kwarg1: true
      });
    }
  </script>
</body>

</html>
'''
  output.eval_js('google.colab.output.setIframeHeight(0, true);')
  output.clear()
  display(HTML(GameHOMEHTML))


def renderSearch():
#searchPage
  SearchHTML=\
'''
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <link rel="stylesheet" href="https://cdnjs.cloudflare.com/ajax/libs/font-awesome/6.2.0/css/all.min.css">
    <link href="https://fonts.googleapis.com/css2?family=Poppins:wght@400;500;600;700&display=swap" rel="stylesheet">
    <title>Search Page</title>
    <style>
        body {
            font-family: 'Poppins', Arial, sans-serif;
            transition: background-color 0.5s ease, color 0.5s ease;
            background-color: rgba(115, 158, 233, 0.4);
            color: #2b2b2b;
            margin: 0;
            padding: 0;
        }

        h1 {
            font-size: 2rem;
            margin-top: 0;
        }

        .button {
          padding: 5px 10px;
          font-size: 12px;
          color: #ffffff;
          border: none;
          border-radius: 5px;
          cursor: pointer;
          margin-right: 8px;
          text-shadow: 1px 1px 2px rgba(0, 0, 0, 0.5);
        }

        header {
            background-color: #1d3a5f;
            padding: 20px;
            color: #ffffff;
            display: flex;
            align-items: center;
            justify-content: space-between;
            border: none;
            border-radius: 0;
        }

        main {
            padding: 20px;
        }

        footer {
            text-align: center;
            background-color: #1d3a5f;
            color: #ffffff;
            padding: 20px;
            position: fixed;
            bottom: 0;
            width: 100%;
        }

        .theme-dark {
            background-color: #1A202C;
            color: #EDF2F7;
        }


        .container {
            max-width: 800px;
            margin: 20px auto;
            padding: 0 20px;
        }

        .search-container {
            text-align: center;
            margin-bottom: 20px;
        }

        .search-input {
            width: 70%;
            padding: 10px;
            font-size: 16px;
            border: 1px solid #ccc;
            border-radius: 5px;
        }

        .search-button {
            padding: 10px 20px;
            font-size: 16px;
            background-color: #007bff;
            color: #fff;
            border: none;
            border-radius: 5px;
            cursor: pointer;
        }

        .search-results {
            list-style-type: none;
            padding: 0;
        }

        .search-result-item {
            margin-bottom: 10px;
        }

        .search-result-title {
            font-size: 18px;
            color: #007bff;
            margin-bottom: 5px;
        }

        .search-result-url {
            font-size: 14px;
            color: #333;
        }


        .text-blue {
            color: #007bff;
        }


        #darkModeToggle {
            font-size: 20px;
        }

        .content {
      flex-grow: 1;
    }

        main {
      padding: 20px;

    }
    .hover\:bg-blue-600:hover {
      background-color: #0056b3;
    }

    .py-2 {
      padding-top: 0.5rem;
      padding-bottom: 0.5rem;
    }

    .rounded-xl {
      border-radius: 0.75rem;
    }

    .shadow-lg {
      box-shadow: 0 4px 6px -1px rgba(0, 0, 0, 0.1), 0 2px 4px -1px rgba(0, 0, 0, 0.06);
    }

    </style>
</head>
    <body>
     <header>
        <h1 class="text-white">EducCloud</h1>
        <div class="flex items-center">
         <a href="javascript:void(0)" class="button mr-2" onclick="GameHOMEHTML()">Home</a>
         <a href="javascript:void(0)" class="button mr-2" onclick="renderAboutUs()">About Us</a>
         <a href="javascript:void(0)" class="button mr-2" onclick="renderContactUs()">Contact Us</a>
         <a href="javascript:void(0)" class="button mr-2" onclick="renderSearch()">Search</a>
         <a href="javascript:void(0)" class="button mr-2" onclick="renderStatistics()">Statistics</a>
         <a href="javascript:void(0)" class="button mr-2" onclick="renderAdmin()">Admin</a>
         <a href="javascript:void(0)" id="logoutButton" class="button flex items-center" onclick="logoutUser(); return confirm('Are you sure you want to log out?');">
          <i class="fas fa-sign-out-alt mr-2"></i> Logout
        </a>
        <button id="darkModeToggle" class="button"><i class="fas fa-moon"></i></button>
      </header>
      <section class="about p-8">
    <main>
    <div class="container">
        <div class="search-container">
            <h1>Search Page</h1>
            <form id="searchForm">
                <input type="text" id="searchInput" class="search-input" placeholder="Enter your search query">
                <button type="submit" class="search-button">Search</button>
            </form>
        </div>
        <ul class="search-results" id="searchResults"></ul>
    </div>
    <main>
    </section>



    <script>
    darkModeToggle.addEventListener('click', () => {
            document.body.classList.toggle('theme-dark');
            if (document.body.classList.contains('theme-dark')) {
                darkModeToggle.innerHTML = '<i class="fas fa-sun"></i>';
            } else {
                darkModeToggle.innerHTML = '<i class="fas fa-moon"></i>';
            }
        })
    </script>

    <footer>
        <p>&copy; 2024 Cloud Computing Learning. All rights reserved.</p>
    </footer>
</body>
</html>
'''

  output.eval_js('google.colab.output.setIframeHeight(0, true);')
  output.clear()
  display(HTML(SearchHTML))


def leaderTableHTML():
# leader table page
  leaderTableHTML = \
'''
<head>
  <meta charset="UTF-8">
  <meta name="viewport" content="width=device-width, initial-scale=1.0">
  <title>Leaders Table</title>
  <style>
    body {
      background-image: url('https://doimages.nyc3.cdn.digitaloceanspaces.com/002Blog/DO-Gaming-Premium-CPUOptimized.png');
      background-size: cover;
      background-repeat: no-repeat;
      background-position: center center;
      font-family: 'CustomFont', sans-serif;
      font-size: 18px;
      height: 570px;
      display: flex;
      align-items: center;
      justify-content: center;
      text-align: center;
      flex-direction: column;
    }
    #screen {
      display: flex;
      flex-direction: column;
      align-items: center;

      justify-content: center;
      padding: 60px;
      border-radius: 10px;
      margin: 0 auto;
      position: absolute;
      top: 50%;
      left: 50%;
      transform: translate(-50%, -50%);
      max-width: 800px;
      width: 100%;
    }

    #title {
      display: inline-block;
      margin-right: 10px;
      font-family: 'CustomFont', sans-serif;
      color: #f2f2f2; /* Adjust the color as needed */
      font-size: 40px;
      text-shadow: 2px 2px 5px black;
    }

    #table-content {
      margin-top: 20px;
      width: 100%;
    }

    #table-content th,
    #table-content td {
      padding: 10px;
      text-align: left;
      color: #000000;
      font-size: 17px;
      text-align: center;
    }

    #table-content th {
      background-color: #624FCD;
    }

    #table-content td {
      background-color: #f2f2f2;
    }

    #backBtn {
      padding: 10px 20px;
      background-color: #624FCD;
      color: white;
      border: none;
      border-radius: 8px;
      font-size: 16px;
      cursor: pointer;
      font-family: 'CustomFont', sans-serif;
      margin-top: 20px;
      text-shadow: 2px 2px 4px rgba(0, 0, 0, 0.5); /* Adjust the shadow values as needed */
      }


    #backBtn:hover {
      background-color: #FF8C00;
    }

    #backBtn:active {
      background-color: #FF8C00;
    }
  </style>
</head>

<body>
<div id='screen'>
  <h1 id='title'>Leader Table</h1>
  <table id='table-content'>
    <tr>
      <th>Place</th>
      <th>Name</th>
      <th>Score</th>
    </tr>
    <!-- Inside the table element -->
    <tr>
      <td>1</td>
      <td>USER2359</td>
      <td>600</td>
    </tr>
    <tr>
      <td>2</td>
      <td>Jane Smith</td>
      <td>550</td>
    </tr>
    <tr>
      <td>3</td>
      <td>USER222</td>
      <td>490</td>
    </tr>
    <tr>
      <td>3</td>
      <td>Bob Johnson</td>
      <td>400</td>
    </tr>
  </table>
  <button id='backBtn'>Back</button>
</div>
<script>
  document.querySelector('#backBtn').onclick = () => {
    google.colab.kernel.invokeFunction('notebook.GameHOMEHTML', []);
  };
</script>
</body>
'''


  output.eval_js('google.colab.output.setIframeHeight(0, true);')
  output.clear()
  display(HTML(leaderTableHTML))

def renderAdmin():
# about us page
  AbdminHTML = \
'''
<!DOCTYPE html>
<html lang="en">
<head>
  <meta charset="UTF-8">
  <meta name="viewport" content="width=device-width, initial-scale=1.0">
  <title>Courses</title>
  <link href="https://cdn.jsdelivr.net/npm/tailwindcss@2.2.19/dist/tailwind.min.css" rel="stylesheet">
  <link rel="stylesheet" href="https://cdnjs.cloudflare.com/ajax/libs/font-awesome/6.5.1/css/all.min.css"
    integrity="sha512-DTOQO9RWCH3ppGqcWaEA1BIZOC6xxalwEsw9c2QQeAIftl+Vegovlnee1c9QX4TctnWMn13TZye+giMm8e2LwA=="
    crossorigin="anonymous" referrerpolicy="no-referrer" />
  <link href="https://fonts.googleapis.com/css2?family=Poppins:wght@400;500;600;700&display=swap" rel="stylesheet">
  <style>
    body {
      font-family: 'Poppins', Arial, sans-serif;
      transition: background-color 0.5s ease, color 0.5s ease;
      background-color: #ffffff;
      color: #1d3a5f;
      margin: 0;
      padding: 0;
    }

    h1 {
      font-size: 2rem;
      margin-top: 0;
    }

    /* Apply the background color only to buttons within main */
     main .button {
      background-color: #1d3a5f;
    }

    /* Additional styles for the buttons within main if needed */
    main .button:hover {
      background-color: #153049; /* Change the hover color if desired */
    }

    main .button:not(:last-child) {
      margin-right: 10px; /* Add margin between buttons */
    }

    .button {
      padding: 10px 20px;
      color: #ffffff;
      border: none;
      border-radius: 5px;
      cursor: pointer;
      margin-right: 10px;
      text-shadow: 1px 1px 2px rgba(0, 0, 0, 0.5);
    }

    .study-now {
      background-color: #1d3a5f;
    }

    header {
      background-color: #1d3a5f;
      padding: 20px;
      color: #ffffff;
      display: flex;
      align-items: center;
      justify-content: space-between;
      border: none;
      border-radius: 0;
    }

    main {
      padding: 20px;
      margin-bottom: 80px;
    }

    h2 {
      font-size: 2rem; /* Increase the font size for "Query Management" */
      margin-bottom: 20px; /* Add spacing below the heading */
    }

    footer {
      text-align: center;
      background-color: #1d3a5f;
      color: #ffffff;
      padding: 20px;
      position: fixed;
      bottom: 0;
      width: 100%;
    }
    .theme-dark {
        background-color: #1A202C;
        color: #EDF2F7;
    }

    .about {
      padding: 8px;
    }

    .row {
      display: flex;
      align-items: center;
    }

    .image img {
      max-width: 100%;
    }

    .content {
      flex-grow: 1;
    }

    .py-2 {
      padding-top: 0.5rem;
      padding-bottom: 0.5rem;
    }

    .rounded-xl {
      border-radius: 0.75rem;
    }

    .shadow-lg {
      box-shadow: 0 4px 6px -1px rgba(0, 0, 0, 0.1), 0 2px 4px -1px rgba(0, 0, 0, 0.06);
    }

    .text-2xl {
      font-size: 1.5rem;
    }

    .fixed {
      position: fixed;
    }

    .bottom-0 {
      bottom: 0;
    }

    .w-full {
      width: 100%;
    }

</style>
</head>
<body>
  <header>
        <h1 class="text-white">EducCloud</h1>
        <div class="flex items-center">
         <a href="javascript:void(0)" class="button mr-2" onclick="GameHOMEHTML()">Home</a>
         <a href="javascript:void(0)" class="button mr-2" onclick="renderAboutUs()">About Us</a>
         <a href="javascript:void(0)" class="button mr-2" onclick="">Contact Us</a>
         <a href="javascript:void(0)" class="button mr-2" onclick="renderSearch()">Search</a>
         <a href="javascript:void(0)" class="button mr-2" onclick="renderStatistics()">Statistics</a>
         <a href="javascript:void(0)" class="button mr-2" onclick="renderAdmin()">Admin</a>
         <a href="javascript:void(0)" id="logoutButton" class="button flex items-center" onclick="logoutUser(); return confirm('Are you sure you want to log out?');">
          <i class="fas fa-sign-out-alt mr-2"></i> Logout
        </a>
        <button id="darkModeToggle" class="button"><i class="fas fa-moon"></i></button>
      </header>

  <main id="mainContent">
    <h2>Query Management</h2>
    <select id="queriesDropdown" class="button">
        <option value="">Select a query</option>
        {% for query in queries %}
        <option value="{{ query }}">{{ query }}</option>
        {% endfor %}
    </select>
    <button onclick="deleteQuery()" class="button">Delete Query</button>
    <div id="linksList"></div>
  </main>


  <footer>
    <div class="footer-content">
      <p>&copy; 2024 Cloud Computing Learning. All rights reserved.</p>
    </div>
  </footer>
  <script>
    $(document).ready(function() {
        $('#queriesDropdown').change(fetchLinks);
    });
    function fetchLinks() {
        var query = $('#queriesDropdown').val();
        if (!query) {
            $('#linksList').empty();
            return;
        }
        $.get(`/get_links/${query}`, function(links) {
            $('#linksList').empty();
            links.forEach(link => {
                $('#linksList').append(`<div>${link} <button onclick="deleteLink('${query}', '${link}')" class="button">Delete Link</button></div>`);
            });
        });
    }
    function deleteQuery() {
        var query = $('#queriesDropdown').val();
        $.post(`/delete_query/${query}`, function() {
            location.reload();
        });
    }
    function deleteLink(query, link) {
        $.ajax({
            url: `/delete_link/${query}`,
            type: 'POST',
            contentType: 'application/json',
            data: JSON.stringify({ link: link }),
            success: function() {
                fetchLinks();
            }
        });
    }
    const darkModeToggle = document.getElementById('darkModeToggle');
    darkModeToggle.addEventListener('click', () => {
        document.body.classList.toggle('theme-dark');
        if (document.body.classList.contains('theme-dark')) {
            darkModeToggle.innerHTML = '<i class="fas fa-sun"></i>';
        } else {
            darkModeToggle.innerHTML = '<i class="fas fa-moon"></i>';
        }
      })
</script>
</body>
</html>
'''

  output.eval_js('google.colab.output.setIframeHeight(0, true);')
  output.clear()
  display(HTML(AbdminHTML))


def renderStatistics():
# statict page
  StatisticsHTML = \
'''
<!DOCTYPE html>
<html lang="en">
<head>
  <meta charset="UTF-8">
  <meta name="viewport" content="width=device-width, initial-scale=1.0">
  <title>Courses</title>
  <link href="https://cdn.jsdelivr.net/npm/tailwindcss@2.2.19/dist/tailwind.min.css" rel="stylesheet">
  <link rel="stylesheet" href="https://cdnjs.cloudflare.com/ajax/libs/font-awesome/6.5.1/css/all.min.css"
    integrity="sha512-DTOQO9RWCH3ppGqcWaEA1BIZOC6xxalwEsw9c2QQeAIftl+Vegovlnee1c9QX4TctnWMn13TZye+giMm8e2LwA=="
    crossorigin="anonymous" referrerpolicy="no-referrer" />
  <link href="https://fonts.googleapis.com/css2?family=Poppins:wght@400;500;600;700&display=swap" rel="stylesheet">
  <style>
    body {
      font-family: 'Poppins', Arial, sans-serif;
      transition: background-color 0.5s ease, color 0.5s ease;
      background-color: #ffffff;
      color: #1d3a5f;
      margin: 0;
      padding: 0;
    }

    h1 {
      font-size: 2rem;
      margin-top: 0;
    }

    /* Apply the background color only to buttons within main */
     main .button {
      background-color: #1d3a5f;
    }

    /* Additional styles for the buttons within main if needed */
    main .button:hover {
      background-color: #153049; /* Change the hover color if desired */
    }

    main .button:not(:last-child) {
      margin-right: 10px; /* Add margin between buttons */
    }

    .button {
      padding: 10px 20px;
      color: #ffffff;
      border: none;
      border-radius: 5px;
      cursor: pointer;
      margin-right: 10px;
      text-shadow: 1px 1px 2px rgba(0, 0, 0, 0.5);
    }

    .study-now {
      background-color: #1d3a5f;
    }

    header {
      background-color: #1d3a5f;
      padding: 20px;
      color: #ffffff;
      display: flex;
      align-items: center;
      justify-content: space-between;
      border: none;
      border-radius: 0;
    }

    main {
      padding: 20px;
      margin-bottom: 80px;
    }

    h2 {
      font-size: 2rem; /* Increase the font size for "Query Management" */
      margin-bottom: 20px; /* Add spacing below the heading */
    }

    footer {
      text-align: center;
      background-color: #1d3a5f;
      color: #ffffff;
      padding: 20px;
      position: fixed;
      bottom: 0;
      width: 100%;
    }
    .theme-dark {
        background-color: #1A202C;
        color: #EDF2F7;
    }

    .about {
      padding: 8px;
    }

    .row {
      display: flex;
      align-items: center;
    }

    .image img {
      max-width: 100%;
    }

    .content {
      flex-grow: 1;
    }

    .py-2 {
      padding-top: 0.5rem;
      padding-bottom: 0.5rem;
    }

    .rounded-xl {
      border-radius: 0.75rem;
    }

    .shadow-lg {
      box-shadow: 0 4px 6px -1px rgba(0, 0, 0, 0.1), 0 2px 4px -1px rgba(0, 0, 0, 0.06);
    }

    .text-2xl {
      font-size: 1.5rem;
    }

    .fixed {
      position: fixed;
    }

    .bottom-0 {
      bottom: 0;
    }

    .w-full {
      width: 100%;
    }



  </style>
</head>
<body>
    <header>
        <h1 class="text-white">EducCloud</h1>
        <div class="flex items-center">
         <a href="javascript:void(0)" class="button mr-2" onclick="GameHOMEHTML()">Home</a>
         <a href="javascript:void(0)" class="button mr-2" onclick="renderAboutUs()">About Us</a>
         <a href="javascript:void(0)" class="button mr-2" onclick="renderContactUs()">Contact Us</a>
         <a href="javascript:void(0)" class="button mr-2" onclick="renderSearch()">Search</a>
         <a href="javascript:void(0)" class="button mr-2" onclick="renderStatistics()">Statistics</a>
         <a href="javascript:void(0)" class="button mr-2" onclick="renderAdmin()">Admin</a>
         <a href="javascript:void(0)" id="logoutButton" class="button flex items-center" onclick="logoutUser(); return confirm('Are you sure you want to log out?');">
          <i class="fas fa-sign-out-alt mr-2"></i> Logout
        </a>
        <button id="darkModeToggle" class="button"><i class="fas fa-moon"></i></button>
    </header>

  <main id="mainContent">
    <div class="container">
        <div id="stats-container">
            <img id="chart-image" src="{{ url_for('static', filename='chart.png') }}" alt="Chart Image">
            <p id="no-data-message" style="display: none;">No data to show</p>
        </div>
    </div>
  </main>


  <footer>
    <div class="footer-content">
      <p>&copy; 2024 Cloud Computing Learning. All rights reserved.</p>
    </div>
  </footer>
  <script>
        document.addEventListener("DOMContentLoaded", function() {
            var chartImage = document.getElementById('chart-image');
            var noDataMessage = document.getElementById('no-data-message');

            chartImage.onload = function() {
                chartImage.style.display = 'block';
                noDataMessage.style.display = 'none';
            };

            chartImage.onerror = function() {
                chartImage.style.display = 'none';
                noDataMessage.style.display = 'block';
            };
        });
        darkModeToggle.addEventListener('click', () => {
        document.body.classList.toggle('theme-dark');
        if (document.body.classList.contains('theme-dark')) {
            darkModeToggle.innerHTML = '<i class="fas fa-sun"></i>';
        } else {
            darkModeToggle.innerHTML = '<i class="fas fa-moon"></i>';
        }
      })
    </script>
</body>
</html>
'''

  output.eval_js('google.colab.output.setIframeHeight(0, true);')
  output.clear()
  display(HTML(StatisticsHTML))





# Register the functions for JavaScript callbacks
output.register_callback('notebook.render_login', render_login)
output.register_callback('notebook.render_sign_up', render_sign_up)
output.register_callback('notebook.render_about_us', render_about_us)
output.register_callback('notebook.renderContactUs', renderContactUs)
output.register_callback('notebook.GameHOMEHTML', GameHOMEHTML)
output.register_callback('notebook.renderSearch', renderSearch)
output.register_callback('notebook.leaderTableHTML', leaderTableHTML)
output.register_callback('notebook.renderAdmin', renderAdmin)
output.register_callback('notebook.renderStatistics', renderStatistics)





# Create an instance of the WelcomePage class and display the welcome page
welcome_page = WelcomePage()
welcome_page.display()
